In [5]:
import requests
import pandas as pd
import pickle
from statistics import mean
import os
import json
import yfinance as yf
import matplotlib.pyplot as plt
import time

In [6]:
def apply_portafolio_theory(tickers_composition,dollar_price,interval,period):
    tickers_statistics={}
    total_invested=sum(x[1]*x[2] for x in  tickers_composition)            
    earnings_df = pd.DataFrame()
    portfolio_earning=0
    for ticker,quantity,current_price in tickers_composition:
        #Getting earnings usgin historicals per ticker 
        hist = yf.Ticker(ticker).history(period=period,interval=interval).dropna()
        hist["earnings"]=(hist.Close-hist.Open)*100/hist.Open
        tickers_statistics[ticker]={ \
                                    "earning":hist.earnings.mean(),\
                                    "risk":hist.earnings.std(),\
                                    "performance":hist.earnings.mean()/hist.earnings.std(),\
                                    "portfolio_percentage":current_price*quantity/total_invested \
                                   }
        earnings_df=pd.concat([earnings_df,hist["earnings"]],axis = 1)
        earnings_df.rename(columns = {'earnings':ticker}, inplace = True)
        #Getting portfolio earning using earnings mean and portfolio percentage per ticker
        portfolio_earning+=tickers_statistics[ticker]['earning']*tickers_statistics[ticker]['portfolio_percentage']

    portfolio_risk=0
    cov_matrix=(earnings_df).cov().values
    
    for x in range(len(earnings_df.cov().values)):
        for y in range(len(earnings_df.cov().values[x])):
            portfolio_risk+=(tickers_composition[x][1]*tickers_composition[x][2]/total_invested)*(tickers_composition[y][1]*tickers_composition[y][2]/total_invested)*cov_matrix[x][y]
    portfolio_risk=portfolio_risk**(1/2)
    portfolio_performance=portfolio_earning/portfolio_risk

#    print(f"Portfolio earnings per {interval}: {portfolio_earning} \
#            \nPortfolio risk per {interval}: {portfolio_risk} \
#            \nPortfolio performance: {portfolio_performance} \
#          ")
    return portfolio_performance,portfolio_earning,portfolio_risk


In [7]:
## S&P500 tickers list
sandp500_list = '''
MMM
AOS
ABT
ABBV
ACN
ADBE
AMD
AES
AFL
A
APD
ABNB
AKAM
ALB
ARE
ALGN
ALLE
LNT
ALL
GOOGL
GOOG
MO
AMZN
AMCR
AMTM
AEE
AEP
AXP
AIG
AMT
AWK
AMP
AME
AMGN
APH
ADI
ANSS
AON
APA
AAPL
AMAT
APTV
ACGL
ADM
ANET
AJG
AIZ
T
ATO
ADSK
ADP
AZO
AVB
AVY
AXON
BKR
BALL
BAC
BAX
BDX
BRK.B
BBY
TECH
BIIB
BLK
BX
BK
BA
BKNG
BWA
BSX
BMY
AVGO
BR
BRO
BF.B
BLDR
BG
BXP
CHRW
CDNS
CZR
CPT
CPB
COF
CAH
KMX
CCL
CARR
CTLT
CAT
CBOE
CBRE
CDW
CE
COR
CNC
CNP
CF
CRL
SCHW
CHTR
CVX
CMG
CB
CHD
CI
CINF
CTAS
CSCO
C
CFG
CLX
CME
CMS
KO
CTSH
CL
CMCSA
CAG
COP
ED
CEG
COO
CPRT
GLW
CPAY
CTVA
CSGP
COST
CTRA
CRWD
CCI
CSX
CMI
CVS
DHR
DRI
DVA
DAY
DECK
DE
DELL
DAL
DVN
DXCM
FANG
DLR
DFS
DG
DLTR
D
DPZ
DOV
DOW
DHI
DTE
DUK
DD
EMN
ETN
EBAY
ECL
EIX
EW
EA
ELV
EMR
ENPH
ETR
EOG
EPAM
EQT
EFX
EQIX
EQR
ERIE
ESS
EL
EG
EVRG
ES
EXC
EXPE
EXPD
EXR
XOM
FFIV
FDS
FICO
FAST
FRT
FDX
FIS
FITB
FSLR
FE
FI
FMC
F
FTNT
FTV
FOXA
FOX
BEN
FCX
GRMN
IT
GE
GEHC
GEV
GEN
GNRC
GD
GIS
GM
GPC
GILD
GPN
GL
GDDY
GS
HAL
HIG
HAS
HCA
DOC
HSIC
HSY
HES
HPE
HLT
HOLX
HD
HON
HRL
HST
HWM
HPQ
HUBB
HUM
HBAN
HII
IBM
IEX
IDXX
ITW
INCY
IR
PODD
INTC
ICE
IFF
IP
IPG
INTU
ISRG
IVZ
INVH
IQV
IRM
JBHT
JBL
JKHY
J
JNJ
JCI
JPM
JNPR
K
KVUE
KDP
KEY
KEYS
KMB
KIM
KMI
KKR
KLAC
KHC
KR
LHX
LH
LRCX
LW
LVS
LDOS
LEN
LLY
LIN
LYV
LKQ
LMT
L
LOW
LULU
LYB
MTB
MPC
MKTX
MAR
MMC
MLM
MAS
MA
MTCH
MKC
MCD
MCK
MDT
MRK
META
MET
MTD
MGM
MCHP
MU
MSFT
MAA
MRNA
MHK
MOH
TAP
MDLZ
MPWR
MNST
MCO
MS
MOS
MSI
MSCI
NDAQ
NTAP
NFLX
NEM
NWSA
NWS
NEE
NKE
NI
NDSN
NSC
NTRS
NOC
NCLH
NRG
NUE
NVDA
NVR
NXPI
ORLY
OXY
ODFL
OMC
ON
OKE
ORCL
OTIS
PCAR
PKG
PLTR
PANW
PARA
PH
PAYX
PAYC
PYPL
PNR
PEP
PFE
PCG
PM
PSX
PNW
PNC
POOL
PPG
PPL
PFG
PG
PGR
PLD
PRU
PEG
PTC
PSA
PHM
QRVO
PWR
QCOM
DGX
RL
RJF
RTX
O
REG
REGN
RF
RSG
RMD
RVTY
ROK
ROL
ROP
ROST
RCL
SPGI
CRM
SBAC
SLB
STX
SRE
NOW
SHW
SPG
SWKS
SJM
SW
SNA
SOLV
SO
LUV
SWK
SBUX
STT
STLD
STE
SYK
SMCI
SYF
SNPS
SYY
TMUS
TROW
TTWO
TPR
TRGP
TGT
TEL
TDY
TFX
TER
TSLA
TXN
TPL
TXT
TMO
TJX
TSCO
TT
TDG
TRV
TRMB
TFC
TYL
TSN
USB
UBER
UDR
ULTA
UNP
UAL
UPS
URI
UNH
UHS
VLO
VTR
VLTO
VRSN
VRSK
VZ
VRTX
VTRS
VICI
V
VST
VMC
WRB
GWW
WAB
WBA
WMT
DIS
WBD
WM
WAT
WEC
WFC
WELL
WST
WDC
WY
WMB
WTW
WYNN
XEL
XYL
YUM
ZBRA
ZBH
ZTS
'''

In [8]:
results=[]
try_tickers=sandp500_list.split()
dollar_price=20.19
interval="1wk"
period="2y"
money_to_invest=8400
tickers_composition=[("AAPL",1),
                     ("AMZN",7) ,\
                     ("CDNS",1) ,\
                     ("GOOGL",20), \
                     ("LLY",1), \
                     ("META",1), \
                     ("MSFT",1), \
                     ("NEE",1), \
                     ("PG",1), \
                     ("TMUS",1), \
                     ("TSLA",1), \
                     ("V",1), \
                     ("VOO",1), \
                     ("XLK",1), \
                     ("CEG",2)
                     
]
ticker_and_price=[ (ticker[0],ticker[1], yf.Ticker(ticker[0]).history(period="1d",interval="1d").iloc[0]['Close']*dollar_price) for ticker in tickers_composition ]

for ticker in try_tickers:
    try:
        print("Using: "+ticker)
        ticker_candidate_price=yf.Ticker(ticker).history(period="1d",interval="1d").iloc[0]['Close']*dollar_price
        if ticker_candidate_price > money_to_invest:
            print(ticker+" is to expensive. Moving to next one")
            continue
        else:
            ticker_and_price.append((ticker, 1,ticker_candidate_price))
            p_perf,p_earn,p_risk=apply_portafolio_theory(ticker_and_price,dollar_price,interval,period)
            results.append((ticker,p_perf,p_earn,p_risk)) 
            ticker_and_price.pop()
            time.sleep(2)
    except :
        print ("Failed with: "+ticker)
        ticker_and_price.pop()
print("-----FINISHED--------")

Using: MMM
Using: AOS
Using: ABT
Using: ABBV
Using: ACN
Using: ADBE
ADBE is to expensive. Moving to next one
Using: AMD
Using: AES
Using: AFL
Using: A
Using: APD
Using: ABNB
Using: AKAM
Using: ALB
Using: ARE
Using: ALGN
Using: ALLE
Using: LNT
Using: ALL
Using: GOOGL
Using: GOOG
Using: MO
Using: AMZN
Using: AMCR
Using: AMTM


AMTM: Period '2y' is invalid, must be one of ['1d', '5d', '1mo', '3mo', 'ytd', 'max']


Using: AEE
Using: AEP
Using: AXP
Using: AIG
Using: AMT
Using: AWK
Using: AMP
AMP is to expensive. Moving to next one
Using: AME
Using: AMGN
Using: APH
Using: ADI
Using: ANSS
Using: AON
Using: APA
Using: AAPL
Using: AMAT
Using: APTV
Using: ACGL
Using: ADM
Using: ANET
Using: AJG
Using: AIZ
Using: T
Using: ATO
Using: ADSK
Using: ADP
Using: AZO
AZO is to expensive. Moving to next one
Using: AVB
Using: AVY
Using: AXON
AXON is to expensive. Moving to next one
Using: BKR
Using: BALL
Using: BAC
Using: BAX
Using: BDX
Using: BRK.B


$BRK.B: possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")


Failed with: BRK.B
Using: BBY
Using: TECH
Using: BIIB
Using: BLK
BLK is to expensive. Moving to next one
Using: BX
Using: BK
Using: BA
Using: BKNG
BKNG is to expensive. Moving to next one
Using: BWA
Using: BSX
Using: BMY
Using: AVGO
Using: BR
Using: BRO


...

Using: GEN
Using: GNRC
Using: GD
Using: GIS
Using: GM
Using: GPC
Using: GILD
Using: GPN
Using: GL
Using: GDDY
Using: GS
GS is to expensive. Moving to next one
Using: HAL
Using: HIG
Using: HAS
Using: HCA
Using: DOC
Using: HSIC
Using: HSY
Using: HES
Using: HPE
Using: HLT
Using: HOLX
Using: HD
HD is to expensive. Moving to next one
Using: HON
Using: HRL
Using: HST
Using: HWM
Using: HPQ
Using: HUBB
HUBB is to expensive. Moving to next one
Using: HUM
Using: HBAN
Using: HII
Using: IBM
Using: IEX
Using: IDXX
IDXX is to expensive. Moving to next one
Using: ITW
Using: INCY
Using: IR
Using: PODD
Using: INTC
Using: ICE
Using: IFF
Using: IP
Using: IPG
Using: INTU
INTU is to expensive. Moving to next one
Using: ISRG
ISRG is to expensive. Moving to next one
Using: IVZ
Using: INVH
Using: IQV
Using: IRM
Using: JBHT
Using: JBL
Using: JKHY
Using: J
Using: JNJ
Using: JCI
Using: JPM
Using: JNPR
Using: K
Using: KVUE
Using: KDP
Using: KEY
Using: KEYS
Using: KMB
Using: KIM
Using: KMI
Using: KKR
Using: KLAC
K

SW: Period '2y' is invalid, must be one of ['1d', '5d', '1mo', '3mo', '6mo', 'ytd', 'max']


Using: SNA
Using: SOLV


SOLV: Period '2y' is invalid, must be one of ['1d', '5d', '1mo', '3mo', '6mo', '1y', 'ytd', 'max']


Using: SO
Using: LUV
Using: SWK
Using: SBUX
Using: STT
Using: STLD
Using: STE
Using: SYK
Using: SMCI
Using: SYF
Using: SNPS
SNPS is to expensive. Moving to next one
Using: SYY
Using: TMUS
Using: TROW
Using: TTWO
Using: TPR
Using: TRGP
Using: TGT
Using: TEL
Using: TDY
TDY is to expensive. Moving to next one
Using: TFX
Using: TER
Using: TSLA
Using: TXN
Using: TPL
TPL is to expensive. Moving to next one
Using: TXT
Using: TMO
TMO is to expensive. Moving to next one
Using: TJX
Using: TSCO
Using: TT
Using: TDG
TDG is to expensive. Moving to next one
Using: TRV
Using: TRMB
Using: TFC
Using: TYL
TYL is to expensive. Moving to next one
Using: TSN
Using: USB
Using: UBER
Using: UDR
Using: ULTA
ULTA is to expensive. Moving to next one
Using: UNP
Using: UAL
Using: UPS
Using: URI
URI is to expensive. Moving to next one
Using: UNH
UNH is to expensive. Moving to next one
Using: UHS
Using: VLO
Using: VTR
Using: VLTO
Using: VRSN
Using: VRSK
Using: VZ
Using: VRTX
VRTX is to expensive. Moving to next one


In [13]:
pd.DataFrame(results).sort_values(by=[2],ascending=False).head(20)

,0,1,2,3
310,NVDA,0.290369,0.805364,2.773592
363,RCL,0.293863,0.805274,2.740308
42,ANET,0.294728,0.803500,2.726242
21,AMZN,0.294946,0.802750,2.721681
422,VST,0.292600,0.802720,2.743400
321,PLTR,0.287803,0.800873,2.782707
122,CRWD,0.285244,0.800516,2.806427
19,GOOG,0.292829,0.800357,2.733194
18,GOOGL,0.292704,0.800158,2.733676
5,AMD,0.291718,0.799657,2.741199


In [ ]:
look_details=('CHKEW', 1, 'NA', 10000)
tickers_composition.append(look_details)
print(apply_portafolio_theory(tickers_composition))
tickers_composition.pop()

(0.15855428460132498, 0.4671408000647136, 2.94625150773636)


('CHKEW', 1, 'NA', 10000)

In [ ]:
tickers_composition

[('AAPL', 1, 2470, 2733.63),
 ('AMZN', 7, 2000, 1670.18),
 ('GOOGL', 20, 2760, 1668.34),
 ('LLY', 1, 3400, 5798.98),
 ('META', 1, 6600, 3304.47),
 ('MSFT', 1, 6000, 4576.11),
 ('NEE', 1, 1500, 1343.98),
 ('PG', 1, 2950, 2525.43),
 ('TMUS', 1, 2567.68, 2568.14),
 ('V', 1, 4420, 3978.76),
 ('VOO', 1, 7571, 6529.05),
 ('XLK', 1, 2282, 2499.29)]